## Day 14

In [1]:
import os
import numpy as np

def read_file(filename="input.txt", path=os.path.join(os.getcwd())):
    with open(os.path.join(path, filename), "r") as f:
        input_list = [line.strip() for line in f]
    return input_list

In [2]:
example = [
    "NNCB",
    "",
    "CH -> B",
    "HH -> N",
    "CB -> H",
    "NH -> C",
    "HB -> C",
    "HC -> B",
    "HN -> C",
    "NN -> C",
    "BH -> H",
    "NC -> B",
    "NB -> B",
    "BN -> B",
    "BB -> N",
    "BC -> B",
    "CC -> N",
    "CN -> C",
]

In [36]:
def preprocess_input(input_list):
    polymer_template = [char for char in input_list[0]]
    
    rules = input_list[2:]
    insertion_rules = []
    for line in rules:
        pair, char = line.split(' -> ')
        insertion_rules.append((pair, char))
    
    return polymer_template, insertion_rules

def find_rule(rules, pair):
    rule = rules[rules[:,0]==pair,1]
    return rule[0]

def separate_pair(rules, pair):
    char = find_rule(rules, pair)
    return pair[0]+char, char+pair[1]

def next_step(count_pairs, rules):
    new_count_pairs = {pair: 0 for pair in count_pairs}
    for pair in count_pairs:
        if count_pairs[pair]:
            pair1, pair2 = separate_pair(rules, pair)
            new_count_pairs[pair1] += count_pairs[pair]
            new_count_pairs[pair2] += count_pairs[pair]
    return new_count_pairs

def count_occurrences(count_pairs, all_chars, first_letter, last_letter):
    occurrences = {char: 0 for char in all_chars}
    for pair in count_pairs:
        c0 = pair[0]
        c1 = pair[1]
        occurrences[c0] += count_pairs[pair]
        occurrences[c1] += count_pairs[pair]
        
    # Cutre but works
    occurrences[first_letter] -= 1
    occurrences[last_letter] -= 1
    occurrences = {char: occurrences[char]//2 for char in occurrences}
    occurrences[first_letter] += 1
    occurrences[last_letter] += 1
    
    return occurrences

In [44]:
template, rules = preprocess_input(read_file())
rules = np.array(rules)

all_chars = np.unique(rules[:, 1])
all_possible_pairs = []
for i in range(len(all_chars)):
    for j in range(len(all_chars)):
        all_possible_pairs.append(all_chars[i]+all_chars[j])
        
count_pairs = {pair: 0 for pair in all_possible_pairs}

polymer = [template[i]+template[i+1] for i in range(len(template)-1)]

for pair in polymer:
    count_pairs[pair] += 1

for _ in range(40):
    count_pairs = next_step(count_pairs, rules)
    
occurrences = count_occurrences(count_pairs, all_chars, template[0], template[-1])

list_occurrences = [(k, v) for k, v in occurrences.items()]
list_occurrences = sorted(list_occurrences, key=lambda x: x[1])

print(f"The result is {list_occurrences[-1][1]} - {list_occurrences[0][1]}" + \
      f" = {list_occurrences[-1][1]-list_occurrences[0][1]}")

The result is 3320320178080 - 887533371027 = 2432786807053
